<a href="https://colab.research.google.com/github/steinjm21/ColabInClass/blob/main/matplotlibProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!git clone https://github.com/steinjm21/NumpyMatplotlib.git

fatal: destination path 'NumpyMatplotlib' already exists and is not an empty directory.


In [40]:
import numpy as np
import sys

In [41]:
def inside_circle(total_count):
    x = np.random.uniform(size=total_count)
    y = np.random.uniform(size=total_count)
    radii = np.sqrt(x*x + y*y)
    count = len(radii[np.where(radii<=1.0)])
    return count

In [44]:
def main():
    n_samples = int(1000000)
    counts = inside_circle(n_samples)
    my_pi = 4.0 * counts / n_samples
    size_of_float = np.dtype(np.float64).itemsize
    memory_required = 3 * n_samples * size_of_float / (1024**3)
    print("Pi: {}, memory: {} GiB".format(my_pi, memory_required))

In [45]:
if __name__ == '__main__':
    main()

Pi: 3.139628, memory: 0.022351741790771484 GiB
